<a href="https://colab.research.google.com/github/lakshmikanthreddybanapuram/QML-Lab/blob/main/qml_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TASK 5: CNOT Gate and Quantum Teleportation**

### **Aim**
To implement, simulate, and verify the functionality of the **Controlled-NOT (CNOT) gate** and demonstrate the **quantum teleportation protocol** using **Qiskit**.

### **1. Mathematical Model of the CNOT Gate**

The **Controlled-NOT (CNOT)** gate is a two-qubit gate where the second qubit (target) is flipped only when the first qubit (control) is in state $|1\rangle$.  
It performs the following transformations:

- $|00\rangle \rightarrow |00\rangle$  
- $|01\rangle \rightarrow |01\rangle$  
- $|10\rangle \rightarrow |11\rangle$  
- $|11\rangle \rightarrow |10\rangle$  

The CNOT gate is used in quantum circuits for entanglement generation and error correction mechanisms.  

### **2. Mathematical Model of Quantum Teleportation**

The **quantum teleportation protocol** allows transmission of an unknown quantum state from one qubit to another distant qubit using entanglement and classical communication.  
The general steps include:

1. **Preparation of Entanglement:**  
   An entangled Bell pair $(|00\rangle + |11\rangle)/\sqrt{2}$ is created between the sender (Alice) and the receiver (Bob).  

2. **Bell Measurement:**  
   Alice performs a joint measurement on her qubits to project them onto one of the Bell states.  

3. **Classical Communication:**  
   Alice sends the two classical bits from her measurement results to Bob.  

4. **Unitary Operations:**  
   Based on the received bits, Bob applies corrective operations ($I$, $X$, $Z$, or $XZ$) to recover the original quantum state.

Mathematically, the teleportation process transfers an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ from Alice’s qubit to Bob’s qubit without physically sending the qubit itself.

### **3. Algorithm – CNOT Gate and Quantum Teleportation**

1. Initialize three qubits:  
   - Qubit 0 holds the state to be teleported.  
   - Qubits 1 and 2 form an entangled Bell pair.  

2. Apply a **Hadamard (H)** gate to qubit 1 and a **CNOT** gate between qubits 1 (control) and 2 (target) to generate entanglement.  

3. Apply **CNOT** between qubits 0 (control) and 1 (target), followed by a **Hadamard (H)** gate on qubit 0.  

4. Measure the first two qubits and send the classical outcomes to the receiver.  

5. Apply corrective gates on qubit 2 according to the classical results:  
   - If the first bit is 1, apply $Z$.  
   - If the second bit is 1, apply $X$.  

6. Verify that the final state of qubit 2 matches the original input state $|\psi\rangle$.



In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


def cnot_circuit(input_state):
    """
    Creates and simulates a CNOT circuit for a given input state.

    Args:
        input_state (str): '00', '01', '10', or '11'
    """
    qc = QuantumCircuit(2, 2)  # 2 qubits, 2 classical bits

    # Prepare input state
    if input_state[0] == '1':
        qc.x(0)  # Set q0 to |1⟩
    if input_state[1] == '1':
        qc.x(1)  # Set q1 to |1⟩

    # Apply CNOT (q0=control, q1=target)
    qc.cx(0, 1)

    # Measure qubits
    qc.measure([0, 1], [0, 1])

    # Simulate
    simulator = Aer.get_backend('qasm_simulator')
    result = simulator.run(qc, shots=1000).result()
    counts = result.get_counts(qc)

    # Plot results
    print(f"\nCNOT Gate Test | Input: |{input_state}⟩")
    print("Circuit Diagram:")
    print(qc.draw(output='text'))
    plot_histogram(counts)
    plt.show()


# Test all possible inputs
for state in ['00', '01', '10', '11']:
    cnot_circuit(state)




from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Create circuit
qc = QuantumCircuit(3, 2)  # 3 qubits, 2 classical bits

# Step 1: Prepare Alice's state (|1⟩ for demo)
qc.x(0)  # Comment out to teleport |0⟩
qc.barrier()

# Step 2: Create Bell pair (q1 & q2)
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# Step 3: Teleportation protocol
qc.cx(0, 1)
qc.h(0)
qc.barrier()

# Step 4: Measure Alice's qubits
qc.measure([0, 1], [0, 1])
qc.barrier()

# Step 5: Bob's corrections
qc.cx(1, 2)  # X if c1=1
qc.cz(0, 2)  # Z if c0=1

# Step 6: Measure Bob's qubit
qc.measure(2, 0)  # Overwrite c0 for verification

# Draw circuit
print("Teleportation Circuit:")
print(qc.draw(output='text'))

# Simulate
simulator = Aer.get_backend('qasm_simulator')
result = simulator.run(qc, shots=1000).result()
counts = result.get_counts(qc)

# Results
print("\nMeasurement results:")
print(counts)
plot_histogram(counts)
plt.show()



CNOT Gate Test | Input: |00⟩
Circuit Diagram:
          ┌─┐   
q_0: ──■──┤M├───
     ┌─┴─┐└╥┘┌─┐
q_1: ┤ X ├─╫─┤M├
     └───┘ ║ └╥┘
c: 2/══════╩══╩═
           0  1 

CNOT Gate Test | Input: |01⟩
Circuit Diagram:
               ┌─┐   
q_0: ───────■──┤M├───
     ┌───┐┌─┴─┐└╥┘┌─┐
q_1: ┤ X ├┤ X ├─╫─┤M├
     └───┘└───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 

CNOT Gate Test | Input: |10⟩
Circuit Diagram:
     ┌───┐     ┌─┐   
q_0: ┤ X ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 

CNOT Gate Test | Input: |11⟩
Circuit Diagram:
     ┌───┐     ┌─┐   
q_0: ┤ X ├──■──┤M├───
     ├───┤┌─┴─┐└╥┘┌─┐
q_1: ┤ X ├┤ X ├─╫─┤M├
     └───┘└───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 
Teleportation Circuit:
     ┌───┐ ░            ░      ┌───┐ ░ ┌─┐    ░            
q_0: ┤ X ├─░────────────░───■──┤ H ├─░─┤M├────░───────■────
     └───┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░       │    
q_1: ──────░─┤ H ├──■───░─┤ 

In [ ]:
pip install qiskit qiskit-aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 89.0 MB/s eta 0:00:00


In [ ]:
pip install qiskit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00


### **4. Result**

The experiment successfully demonstrated the implementation, simulation, and verification of the **CNOT gate** using **Qiskit**, followed by the construction of a complete **quantum teleportation protocol**. The simulation validated the correct operation of the CNOT gate by observing the expected transformations of all possible input states.  

The quantum teleportation circuit accurately transferred an arbitrary quantum state from Alice to Bob using entanglement and classical communication. The results confirm that the protocol enables the faithful reconstruction of the original qubit at Bob’s end, illustrating one of the most fundamental applications of quantum information theory.
